In [ ]:
import matplotlib.pyplot as plt
import time
from IPython import display
from xml.dom import minidom
import math 

In [ ]:
#READ GPX FILE
data=open('./input/Braassem_drecht_18_k.gpx')
# data=open('./../tracks/uitgeest_8_k.gpx')
xmldoc = minidom.parse(data)
track = xmldoc.getElementsByTagName('trkpt')
elevation=xmldoc.getElementsByTagName('ele')
datetime=xmldoc.getElementsByTagName('time')
n_track=len(track)

In [ ]:
#PARSING GPX ELEMENT
lon_list=[]
lat_list=[]
h_list=[]
time_list=[]
for s in range(n_track):
    lon,lat=track[s].attributes['lon'].value,track[s].attributes['lat'].value
    elev=elevation[s].firstChild.nodeValue
    lon_list.append(float(lon))
    lat_list.append(float(lat))
    h_list.append(float(elev))
    # # PARSING TIME ELEMENT
    # dt=datetime[s].firstChild.nodeValue
    # time_split=dt.split('T')
    # hms_split=time_split[1].split(':')
    # time_hour=int(hms_split[0])
    # time_minute=int(hms_split[1])
    # # time_second=int(hms_split[2].split('Z')[0])
    # time_second=int(hms_split[2].split('Z')[0].split('.')[0])
    # total_second=time_hour*3600+time_minute*60+time_second
    # time_list.append(total_second)

In [ ]:
#GEODETIC TO CARTERSIAN FUNCTION
def geo2cart(lon,lat,h):
    a=6378137 #WGS 84 Major axis
    b=6356752.3142 #WGS 84 Minor axis
    e2=1-(b**2/a**2)
    N=float(a/math.sqrt(1-e2*(math.sin(math.radians(abs(lat)))**2)))
    X=(N+h)*math.cos(math.radians(lat))*math.cos(math.radians(lon))
    Y=(N+h)*math.cos(math.radians(lat))*math.sin(math.radians(lon))
    return X,Y

#DISTANCE FUNCTION
def distance(x1,y1,x2,y2):
    d=math.sqrt((x1-x2)**2+(y1-y2)**2)
    return d

#SPEED FUNCTION
def speed(x0,y0,x1,y1,t0,t1):
    d=math.sqrt((x0-x1)**2+(y0-y1)**2)
    delta_t=t1-t0
    s=float(d/delta_t)
    return s

In [ ]:
import pyproj

# Function to convert Geodetic coordinates to Cartesian
def geodetic_to_cartesian(lat, lon, alt):
    # Define the coordinate systems
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    # Perform the coordinate transformation
    x, y, z = pyproj.transform(lla, ecef, lon, lat, alt, radians=False)
    return x, y, z

In [ ]:
#POPULATE DISTANCE AND SPEED LIST
d_list=[0.0]
speed_list=[0.0]
l=0
for k in range(n_track-1):
    if k<(n_track-1):
        l=k+1
    else:
        l=k
    XY0=geo2cart(lon_list[k],lat_list[k],h_list[k])
    XY1=geo2cart(lon_list[l],lat_list[l],h_list[l])
    # XY0=geodetic_to_cartesian(lon=lon_list[k], lat=lat_list[k], alt=h_list[k])
    # XY1=geodetic_to_cartesian(lon=lon_list[l], lat=lat_list[l], alt=h_list[l])

    #DISTANCE
    d=distance(XY0[0],XY0[1],XY1[0],XY1[1])
    sum_d=d+d_list[-1]
    d_list.append(sum_d)
    
    #SPEED
    # s=speed(XY0[0],XY0[1],XY1[0],XY1[1],time_list[k],time_list[l])
    # speed_list.append(s)

In [ ]:
d_list[-1]

zowel geo2cart als geodetic_to_cartesian komen op 7541 meter uit voor het Uitgeest track.
Volgens garmin (en strava) is het track 8,19 km. dus waar zit het verschil?

code: https://www.geodose.com/2018/04/create-gpx-tracking-file-visualizer-python.html


voor Braassem_drecht komt er 16846 meter uit (tegen 18,27 km).

In [ ]:
#PLOT TRACK
f,(track,speed,elevation)=plt.subplots(3,1)
f.set_figheight(8)
f.set_figwidth(5)
plt.subplots_adjust(hspace=0.5)
track.plot(lon_list,lat_list,'k')
track.set_ylabel("Latitude")
track.set_xlabel("Longitude")
track.set_title("Track Plot")

# #PLOT SPEED
# speed.bar(d_list,speed_list,30,color='w',edgecolor='w')
# speed.set_title("Speed")
# speed.set_xlabel("Distance(m)")
# speed.set_ylabel("Speed(m/s)")

#PLOT ELEVATION PROFILE
base_reg=0
elevation.plot(d_list,h_list)
elevation.fill_between(d_list,h_list,base_reg,alpha=0.1)
elevation.set_title("Elevation Profile")
elevation.set_xlabel("Distance(m)")
elevation.set_ylabel("GPS Elevation(m)")
elevation.grid()

In [ ]:
import gpxpy
import gpxpy.gpx

# Parsing an existing file:
# -------------------------

gpx_file = open('./input/Braassem_drecht_18_k.gpx', 'r')

gpx = gpxpy.parse(gpx_file)

In [ ]:
for track in gpx.tracks:
    # for segment in track.segments:
    #     for point in segment.points:
    #         print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevation}')

    print('length 2d: ' + str(track.length_2d()))
    print('length 3d: ' + str(track.length_3d()))


In [ ]:
gpx.tracks[0].length_2d()

In [ ]:
for track in gpx.tracks:
    for segment in track.segments:
        print('segment')
        for point in segment.points:
            print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevation}')

In [ ]:
from gpxpy import geo as mod_geo


d_list = [0.0]
for track in gpx.tracks:
    for segment in track.segments:
        l=0
        for k in range(len(segment.points)-1):
            if k<(len(segment.points)-1):
                l=k+1
            else:
                l=k
            # d = mod_geo.distance(segment.points[k].latitude, segment.points[k].longitude, None, segment.points[l].latitude, segment.points[l].longitude, None)
            d = mod_geo.haversine_distance(segment.points[k].latitude, segment.points[k].longitude, segment.points[l].latitude, segment.points[l].longitude)
            sum_d=d+d_list[-1]
            d_list.append(sum_d)


print(d_list[-1])

gpxpy length_2d() komt op ongeveer 200 meter teveel uit
Uitgeest gpxpy 8356. garmin/strava 8,19km
Braassem gpxpy 18452. garmin/strava 18,27km

18452.49113139704 (distance)
18452.491047374424 (haversine)

In [ ]:
pd = gpx.get_points_data()

pd[-1]

new_gpx = gpxpy.gpx.GPX()
new_gpx.description = "Modified GPX;blabla"

print(new_gpx.to_xml())